# Reactant demo

In [ ]:
using Tenet
using EinExprs
using Reactant
using Enzyme
using BenchmarkTools
using Adapt

using LinearAlgebra
BLAS.set_num_threads(1)

using Random
Random.seed!(0)

In [ ]:
tn = rand(TensorNetwork, 15, 3; dim=(16,16))

path = einexpr(tn; optimizer=Exhaustive())

@benchmark contract(tn; path)

In [ ]:
tn′ = adapt(Reactant.ConcreteRArray, tn)

g = Reactant.compile(Tuple(tensors(tn′))) do ts...
    _tn = TensorNetwork(ts)
    contract(_tn; path)
end

@benchmark g(tensors(tn′)...)

In [ ]:
function f(x...)
    _tn = TensorNetwork(x)
    contract(_tn; path)
end

∇g = Reactant.compile(Tuple(tensors(tn′))) do x...
    dx = Enzyme.make_zero.(x)
    Enzyme.autodiff(Reverse, f, Active, Duplicated.(x,dx)...)
    return dx
end

@benchmark ∇g(tensors(tn′)...)

In [ ]:
re∇g = Reactant.compile(Tuple(tensors(tn′))) do x...
    dx = Enzyme.make_zero.(x)
    primal = Enzyme.autodiff(ReverseWithPrimal, f, Active, Duplicated.(x,dx)...)
    return (primal, dx)
end

@benchmark re∇g(tensors(tn′)...)